## Original Example - Theano

In [3]:
import os
os.chdir('../')
import pymc3 as pm
import numpy as np
pm.__file__

'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [4]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [5]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [beta]
>Metropolis: [alpha]
100%|██████████| 5500/5500 [00:01<00:00, 2869.12it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.905453
[ 0.9458999   2.60061359]


In [1]:
#basic_model2.vars

## Original Example - TF

In [1]:
import os
os.chdir('../')
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm
import numpy as np
pm.__file__

Using TensorFlow backend.


'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    
    # requires initial value now. 
    # also has to be float at the moment. 
    alpha = pm.Normal(name='alpha',initial_value=0.0, mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',initial_value=np.array([1.0,1.0],dtype='float32'), mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',initial_value=np.ones_like(Y), mu=mu, sd=1.0, observed=Y)

    step = pm.Metropolis()
    trace = pm.sample(5000,step=step,cores=1,chains=1)

print(np.mean(trace['alpha:0']))
print(np.mean(trace['beta:0'],axis=0))

Sequential sampling (1 chains in 1 job)
CompoundStep
>Metropolis: [<tf.Variable 'alpha:0' shape=() dtype=float32_ref>]
>Metropolis: [<tf.Variable 'beta:0' shape=(2,) dtype=float32_ref>]
100%|██████████| 5500/5500 [00:12<00:00, 454.51it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.905485
[ 0.94781202  2.62093997]


## Just evaluating the gradient of the prior

### Theano 

In [1]:
import os
os.chdir('../')
import pymc3 as pm
pm.__file__

Using Theano backend.


'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [2]:
# Do I need data to evaluate the gradient?


import numpy as np
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
model = pm.Model()
with model:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0, shape=2)
    
    #step = pm.NUTS()
    #trace = pm.sample(5000,step=step,cores=1,chains=1)
    
# this is just the prior model!!

In [4]:
model.vars[0].name

'alpha'

In [5]:
str(model.vars[0])

'alpha'

In [6]:
# print(np.mean(trace['alpha']))
# print(np.mean(trace['beta'],axis=0))

In [7]:
value_grad_function = model.logp_dlogp_function([alpha,beta])
# this calls 
# ValueGradFunction(self.logpt, grad_vars, extra_vars, **kwargs)
# self.logpt becomes self.cost for the ValueGradFunction

# _build_joined is a bit tricky to understand
# S.clone(cost, replace=replace) replaces the the input variables in the computation graph for cost. 
# which are just now in a vector 
value_grad_function

In [8]:
value_grad_function._extra_are_set = True
value_grad_function(array=np.array([0.1,1.0,1.0]),extra_vars=None)
#array = {'alpha':0.1,'beta'}

(array(-9.67462158203125, dtype=float32),
 array([-0.001, -0.01 , -0.01 ], dtype=float32))

### TensorFlow

In [1]:
import os
os.chdir('../')
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm
pm.__file__

Using TensorFlow backend.


'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [2]:
import numpy as np
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
model = pm.Model()
with model:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0,initial_value=0.0) 
    beta = pm.Normal(name='beta', mu=0.0, sd=10.0,
                     initial_value=np.array([1.0,1.0],dtype='float32'), 
                     shape=2) # this is super annoying that you have to specify the dtype like this.  
    

In [4]:
model.free_RVs[0].name


'alpha:0'

In [5]:
value_grad_function = model.logp_dlogp_function([alpha,beta])
# this calls 
# ValueGradFunction(self.logpt, grad_vars, extra_vars, **kwargs)
# self.logpt becomes self.cost for the ValueGradFunction

# _build_joined is a bit tricky to understand
# S.clone(cost, replace=replace) replaces the the input variables in the computation graph for cost. 
# which are just now in a vector 
value_grad_function

In [6]:
value_grad_function

In [7]:
value_grad_function._extra_are_set = True
value_grad_function(array=np.array([0.1,1.0,1.0]),extra_vars=None)

(-9.6746216, array([-0.001, -0.01 , -0.01 ], dtype=float32))

## NUTS - Theano

In [1]:
import os
os.chdir('../')
import pymc3 as pm
import numpy as np
pm.__file__

Using Theano backend.


'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    alpha = pm.Normal(name='alpha',mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',mu=mu, sd=1.0, observed=Y)

    step = pm.NUTS()
    trace = pm.sample(5000,step=step,cores=1,chains=1) # switched off parallel for simplifications. 

print(np.mean(trace['alpha']))
print(np.mean(trace['beta'],axis=0))

Sequential sampling (1 chains in 1 job)
NUTS: [beta, alpha]
100%|██████████| 5500/5500 [00:04<00:00, 1165.45it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


0.909692
[ 0.94828242  2.61490655]


## NUTS - TF

- need to get rid of the ancestor stuff - put it into backend.

In [1]:
import os
os.chdir('../')
os.environ["PYMC_SYMB_BACKEND"] = "tensorflow" 
import pymc3 as pm
import numpy as np
pm.__file__

Using TensorFlow backend.


'/Users/chris/Desktop/pymc3/pymc3/__init__.py'

In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1; beta = [1, 2.5]

# Predictor variable
X1 = np.random.randn(100)
X2 = np.random.randn(100) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(100)*sigma
Y=Y.astype('float32')

In [3]:
basic_model2 = pm.Model()
with basic_model2:
    
    # requires initial value now. 
    # also has to be float at the moment. 
    alpha = pm.Normal(name='alpha',initial_value=0.0, mu=0.0, sd=10.0) 
    beta = pm.Normal(name='beta',initial_value=np.array([1.0,1.0],dtype='float32'), mu=0.0, sd=10.0, shape=2)

    mu = alpha + beta[0]*X1 + beta[1]*X2

    Y_obs = pm.Normal(name='Y_obs',initial_value=np.ones_like(Y), mu=mu, sd=1.0, observed=Y)

    step = pm.NUTS()
    trace = pm.sample(5000,step=step,cores=1,chains=1)

print(np.mean(trace['alpha:0']))
print(np.mean(trace['beta:0'],axis=0))

AttributeError: 'FreeRV' object has no attribute 'owner'